In [2]:
from langchain.chat_models import init_chat_model
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]  # plural "messages"

In [4]:
@tool
def get_stock_price(symbol: str) -> float:
    '''Return the current price of a stock given the stock symbol'''
    return {"MSFT": 200.3, "AAPL": 100.4, "AMZN": 150.0, "RIL": 87.6}.get(symbol, 0.0)

In [5]:
@tool
def buy_stocks(symbol: str, quantity: int, total_price: float) -> str:
    '''Buy stocks given the stock symbol and quantity'''
    decision = interrupt(f"Approve buying {quantity} {symbol} stocks for ${total_price:.2f}?")

    if decision == "yes":
        return f"You bought {quantity} shares of {symbol} for a total price of {total_price}"
    else:
        return "Buying declined."

In [6]:
tools=[get_stock_price, buy_stocks]

In [7]:
llm = init_chat_model("openai:gpt-4.1-mini")
llm_with_tools = llm.bind_tools(tools)

In [13]:
def chatbot_node(state: State):
    return {"messages" : [llm_with_tools.invoke(state["messages"])]}

In [14]:
memory = MemorySaver()

builder = StateGraph(State)
builder.add_node("chatbot", chatbot_node)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile(checkpointer=memory)


In [15]:
config = {"configurable": {"thread_id": "buy_thread"}}

In [16]:
# Step 1: user asks price
state = graph.invoke({"messages":[{"role":"user","content":"What is the current price of 10 MSFT stocks?"}]}, config=config)
print(state["messages"][-1].content)

The current price of one MSFT stock is $200.3. For 10 MSFT stocks, the total price would be $2003. Would you like to proceed with buying 10 MSFT stocks?


In [ ]:
# Step 2: user asks to buy
state = graph.invoke({"messages":[{"role":"user","content":"Buy 10 MSFT stocks at current price."}]}, config=config)
print(state.get("__interrupt__"))
print("-- " * 10)
print(state)

[Interrupt(value='Approve buying 10 MSFT stocks for $2003.00?', resumable=True, ns=['tools:c236f008-7c18-659f-a23f-1d12b762820c'])]


In [18]:
decision = input("Approve (yes/no): ")
state = graph.invoke(Command(resume=decision), config=config)
print(state["messages"][-1].content)

You have successfully bought 10 MSFT stocks at the current price, with a total cost of $2003. Is there anything else you would like to do?
